In [5]:
import numpy as np

from ase.io import read
atoms=read('data.vasp',format='vasp')

#all atom is in central region by definition
#here we assume number of ghost shell is 1

#construct the list of vector connecting between each atoms

masses=atoms.get_masses()
natom=len(atoms.positions)

dist=np.zeros((natom,natom,3))
for i in range(natom):
    for j in range(i):
        rmin=np.zeros(3)
        rij=atoms.get_distance(i,j,vector=True)
        rij_mic=atoms.get_distance(i,j,mic=True,vector=True)
        if(np.linalg.norm(rij_mic)<np.linalg.norm(rij)):
            rmin=rij_mic
        else:
            rmin=rij
        dist[i,j]=rmin


In [6]:
with open('FORCE_CONSTANTS_2ND','r') as fc:
    lines=fc.readlines()

    nlineblock=4
    fc_all=np.zeros((natom,natom,3,3))
    start=1
    for i in range(natom):
        for j in range(natom):
            fc_block=lines[start+1:start+nlineblock]
            fc=np.loadtxt(fc_block)
            fc_all[i,j]=fc/np.sqrt(masses[i]*masses[j])
            #fc_all[i,j]=fc
            start=start+nlineblock

In [7]:
Rx=dist[:,:,0]
Ry=dist[:,:,1]
Rz=dist[:,:,2]

Vx=np.zeros((natom,natom,3,3))
Vy=np.zeros((natom,natom,3,3))
Vz=np.zeros((natom,natom,3,3))

#loop only for lower triangle
#make sure that Vij=-Vji
for i in range(natom):
    for j in range(i):
        Vijx=-Rx[i,j]*fc_all[i,j]
        Vx[i,j]=Vijx
        Vx[j,i]=-Vijx

        Vijy=-Ry[i,j]*fc_all[i,j]
        Vy[i,j]=Vijy
        Vy[j,i]=-Vijy

        Vijz=-Rz[i,j]*fc_all[i,j]
        Vz[i,j]=Vijz
        Vz[j,i]=-Vijz

In [8]:
flatVx=np.reshape(Vx.transpose(0,2,1,3),(24,24))
GULP_Vijx=np.loadtxt('Vijx.dat').reshape((24,24))
np.where(np.abs(flatVx-GULP_Vijx) > 0.001)

(array([12, 12, 13, 13, 14, 14, 15, 15, 16, 16, 17, 17, 18, 18, 19, 19, 20,
        20, 21, 21, 22, 22, 23, 23]),
 array([18, 21, 19, 22, 20, 23, 18, 21, 19, 22, 20, 23, 12, 15, 13, 16, 14,
        17, 12, 15, 13, 16, 14, 17]))

In [9]:
for i in range(natom):
    for j in range(i):
        print (i+1,j+1,dist[i,j,0])

2 1 0.0
3 1 2.7249999999999996
3 2 2.7249999999999996
4 1 2.7249999999999996
4 2 2.7249999999999996
4 3 0.0
5 1 -1.3625000000000007
5 2 -1.3625000000000007
5 3 1.3625000000000003
5 4 1.3625000000000003
6 1 -1.3625000000000007
6 2 -1.3625000000000007
6 3 1.3625000000000003
6 4 1.3625000000000003
6 5 0.0
7 1 1.3624999999999998
7 2 1.3624999999999998
7 3 -1.3624999999999998
7 4 -1.3624999999999998
7 5 -2.725
7 6 -2.725
8 1 1.3624999999999998
8 2 1.3624999999999998
8 3 -1.3624999999999998
8 4 -1.3624999999999998
8 5 -2.725
8 6 -2.725
8 7 0.0
